In [9]:
from pyspark.sql import SparkSession

In [8]:
spark = SparkSession.builder.appName('cruise').getOrCreate()

In [7]:
df = spark.read.csv("/Users/david/gitProjects/Sparkmllib/cruise_ship_info.csv",inferSchema=True, header=True)
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [11]:
for ship in df.head(5):
    print(ship)

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)
Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)
Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7)
Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1)
Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0)


In [13]:
df.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [14]:
from pyspark.ml.feature import StringIndexer

In [19]:
indexer = StringIndexer(inputCol='Cruise_line', outputCol='cruise_cat')
indexed = indexer.fit(df).transform(df)
indexed.head(3)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, cruise_cat=1.0)]

In [20]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler


In [23]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_cat']

In [25]:
assembler = VectorAssembler(inputCols=['Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'cruise_cat'],outputCol = 'features')


In [26]:
output = assembler.transform(indexed)

In [28]:
output.select('features','crew').show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[30.2769999999999...|3.55|
|[30.2769999999999...|3.55|
|[47.262,14.86,7.2...| 6.7|
|[110.0,29.74,9.53...|19.1|
|[101.353,26.42,8....|10.0|
|[70.367,20.52,8.5...| 9.2|
|[70.367,20.52,8.5...| 9.2|
|[70.367,20.56,8.5...| 9.2|
|[70.367,20.52,8.5...| 9.2|
|[110.238999999999...|11.5|
|[110.0,29.74,9.51...|11.6|
|[46.052,14.52,7.2...| 6.6|
|[70.367,20.52,8.5...| 9.2|
|[70.367,20.52,8.5...| 9.2|
|[86.0,21.24,9.63,...| 9.3|
|[110.0,29.74,9.51...|11.6|
|[88.5,21.24,9.63,...|10.3|
|[70.367,20.52,8.5...| 9.2|
|[88.5,21.24,9.63,...| 9.3|
|[70.367,20.52,8.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [29]:
data = output.select('features','crew')

In [33]:
train,test = data.randomSplit([0.7,0.3])

In [32]:
train.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               105|
|   mean|7.7020952380952385|
| stddev|3.5548300552363576|
|    min|              0.59|
|    max|              19.1|
+-------+------------------+



In [35]:
test.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|               46|
|   mean|8.143043478260871|
| stddev|4.197247176988068|
|    min|             0.88|
|    max|             21.0|
+-------+-----------------+



In [36]:
from pyspark.ml.regression import LinearRegression

In [38]:
lr = LinearRegression(labelCol='crew')

In [40]:
model = lr.fit(train)

In [42]:
 results = model.evaluate(test)

In [44]:
results.rootMeanSquaredError

1.3544218340989131

In [45]:
results.r2

0.8935552811932739

In [50]:
results.meanAbsoluteError

0.7580265721617978

In [49]:
results.meanSquaredError

1.8344585046838637

In [52]:
from pyspark.sql.functions import corr


In [55]:
df.select(corr('crew','passengers')).show() # or crew cabins 

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+

